In [5]:
import pandas as pd
import censusdata
import csv
import requests
import us
import zipfile

from pathlib import Path

DATA_PATH = Path.cwd().parent / "data"
FIPS_CSV_PATH = DATA_PATH / "fips_states_2010.csv"
OUTPUT_PATH = DATA_PATH / "dataset" / "lead_score"

GEOID_FIELD_NAME = "GEOID10"

In [11]:
# See data at https://data.openei.org/submissions/573

state_abbreviations = [state.abbr for state in us.states.STATES_AND_TERRITORIES]

# Download each state / territory individually
dfs = []
for state_abbreviation in state_abbreviations:
    print(f"Downloading data for state/territory with abbreviation {state_abbreviation}")

    download = requests.get(
        f"https://data.openei.org/files/573/{state_abbreviation}-2018-LEAD-data.zip",
        verify=False,
    )
    file_contents = download.content
    zip_file_dir = DATA_PATH / "tmp" / "lead_score"

    # Make the directory if it doesn't exist
    zip_file_dir.mkdir(parents=True, exist_ok=True)
    zip_file_path = zip_file_dir / f"{state_abbreviation}-downloaded.zip"
    zip_file = open(zip_file_name, "wb")
    zip_file.write(file_contents)
    zip_file.close()

    with zipfile.ZipFile(zip_file_name, "r") as zip_ref:
        zip_ref.extractall(zip_file_dir)

    # New file name:
    tmp_csv_file_path = zip_file_dir / f"htaindex_data_blkgrps_{fips}.csv"
    tmp_df = pd.read_csv(filepath_or_buffer=tmp_csv_file_path)

    dfs.append(tmp_df)

df = pd.concat(dfs)

df.head()

/Users/lucas/.virtualenvs/scoring/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.openei.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


NameError: name 'fips' is not defined

In [ ]:
# Rename and reformat block group ID
df.rename(columns={"blkgrp": GEOID_FIELD_NAME}, inplace=True)
df[GEOID_FIELD_NAME] = df[GEOID_FIELD_NAME].str.replace('"', "")

In [ ]:
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

df.to_csv(path_or_buf=OUTPUT_PATH / "usa.csv", index=False)